<a href="https://colab.research.google.com/github/pkrc267/IndoorLocalization/blob/master/20_04_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pandas.testing as tm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import datasets
from io import StringIO

from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:

frame0=pd.read_csv("/content/newRssiDblock-4.csv")

## Reading processed data without **'Z'** attribute

In [33]:
frame0.head()

,Name,Address,Signal Level,X,Y,MAC,Location
0,BPGC-NAB,70:EA:1A:22:8F:E0,-41,20,26,124150763130848,2026
1,BPGC-NAB,08:4F:A9:10:4B:80,-49,20,26,9138231856000,2026
2,BPGC-NAB,00:A3:8E:60:F2:A0,-72,20,26,702468387488,2026
3,BPGC-WIFI,40:01:7A:B1:00:40,-70,20,26,70375097565248,2026
4,BPGC-NAB,70:EA:1A:22:A9:A0,-66,20,26,124150763137440,2026




---
# Feature Engineering

### Filtering out too weak signal strengths


In [34]:
location_signalMean = frame0.groupby('Location', as_index=False)['Signal Level'].mean()
type(location_signalMean)

pandas.core.frame.DataFrame

In [0]:
lsm = location_signalMean.copy()

In [36]:
frame0.shape

(1691, 7)

In [0]:
for i, r1 in lsm.iterrows():
  for j, r2 in frame0.iterrows():
    if r2['Location'] == r1['Location']:
      if (r2['Signal Level'] < (r1['Signal Level'] - 10)):
      # if (r2['Signal Level'] < (r1['Signal Level'] - 20)) or (r2['Signal Level'] > (r1['Signal Level'] + 20)):
        # print(r2['Signal Level'], "------", r1['Signal Level'])
        frame0.drop(j, inplace=True)

In [45]:
frame0.shape

(1222, 7)

### Performing **_One-Hot Encoding_** to create new features out of "*Categorical data*"

In [0]:
frame = pd.get_dummies(frame0)

In [47]:
frame.head()

,Signal Level,X,Y,MAC,Location,Name_BPGC-LT,Name_BPGC-NAB,Name_BPGC-WIFI,Name_CRT-MIFI,Name_DIRECT-1D-HP DeskJet 2600 series,Name_SAIMAA,Name_TG,Name_bhikkhu,Name_sks,Address_00:A3:8E:4E:29:A0,Address_00:A3:8E:60:F2:A0,Address_04:95:E6:82:45:88,Address_08:4F:A9:10:4B:80,Address_08:4F:A9:2D:77:40,Address_08:4F:A9:2D:8A:40,Address_08:4F:A9:2D:8A:A0,Address_08:4F:A9:30:00:00,Address_08:4F:A9:39:68:20,Address_08:4F:A9:39:68:40,Address_08:4F:A9:39:6A:40,Address_08:4F:A9:43:38:A0,Address_40:01:7A:01:B8:80,Address_40:01:7A:01:B8:8F,Address_40:01:7A:AF:D3:60,Address_40:01:7A:B1:00:40,Address_40:01:7A:C6:A5:60,Address_40:01:7A:CA:3F:AE,Address_60:6D:C7:A3:30:E3,Address_68:3B:78:E9:C0:E0,Address_70:EA:1A:22:87:60,Address_70:EA:1A:22:8F:E0,Address_70:EA:1A:22:92:40,Address_70:EA:1A:22:96:00,Address_70:EA:1A:22:96:20,Address_70:EA:1A:22:96:A0,Address_70:EA:1A:22:97:80,Address_70:EA:1A:22:98:20,Address_70:EA:1A:22:9B:80,Address_70:EA:1A:22:9B:E0,Address_70:EA:1A:22:9F:20,Address_70:EA:1A:22:9F:80,Address_70:EA:1A:22:9F:E0,Address_70:EA:1A:22:A0:20,Address_70:EA:1A:22:A5:A0,Address_70:EA:1A:22:A9:40,Address_70:EA:1A:22:A9:A0,Address_70:EA:1A:22:AC:00,Address_80:14:A8:BE:BB:28,Address_88:63:DF:BB:74:A5,Address_8C:21:A7:E1:4E:22,Address_A8:B4:56:CA:8F:40,Address_A8:B4:56:CC:DE:00,Address_AC:E2:D3:B6:BA:1E
0,-41,20,26,124150763130848,2026,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-49,20,26,9138231856000,2026,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-72,20,26,702468387488,2026,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-70,20,26,70375097565248,2026,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-66,20,26,124150763137440,2026,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### new features help in stronger prediction



In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
TV=frame.drop(['X','Y'], axis=1)
FV=frame.Location
TV.fillna(TV.mean, inplace=True)

Filled NaN entries with **mean** of corresponding columns

In [50]:
TV.head()

,Signal Level,MAC,Location,Name_BPGC-LT,Name_BPGC-NAB,Name_BPGC-WIFI,Name_CRT-MIFI,Name_DIRECT-1D-HP DeskJet 2600 series,Name_SAIMAA,Name_TG,Name_bhikkhu,Name_sks,Address_00:A3:8E:4E:29:A0,Address_00:A3:8E:60:F2:A0,Address_04:95:E6:82:45:88,Address_08:4F:A9:10:4B:80,Address_08:4F:A9:2D:77:40,Address_08:4F:A9:2D:8A:40,Address_08:4F:A9:2D:8A:A0,Address_08:4F:A9:30:00:00,Address_08:4F:A9:39:68:20,Address_08:4F:A9:39:68:40,Address_08:4F:A9:39:6A:40,Address_08:4F:A9:43:38:A0,Address_40:01:7A:01:B8:80,Address_40:01:7A:01:B8:8F,Address_40:01:7A:AF:D3:60,Address_40:01:7A:B1:00:40,Address_40:01:7A:C6:A5:60,Address_40:01:7A:CA:3F:AE,Address_60:6D:C7:A3:30:E3,Address_68:3B:78:E9:C0:E0,Address_70:EA:1A:22:87:60,Address_70:EA:1A:22:8F:E0,Address_70:EA:1A:22:92:40,Address_70:EA:1A:22:96:00,Address_70:EA:1A:22:96:20,Address_70:EA:1A:22:96:A0,Address_70:EA:1A:22:97:80,Address_70:EA:1A:22:98:20,Address_70:EA:1A:22:9B:80,Address_70:EA:1A:22:9B:E0,Address_70:EA:1A:22:9F:20,Address_70:EA:1A:22:9F:80,Address_70:EA:1A:22:9F:E0,Address_70:EA:1A:22:A0:20,Address_70:EA:1A:22:A5:A0,Address_70:EA:1A:22:A9:40,Address_70:EA:1A:22:A9:A0,Address_70:EA:1A:22:AC:00,Address_80:14:A8:BE:BB:28,Address_88:63:DF:BB:74:A5,Address_8C:21:A7:E1:4E:22,Address_A8:B4:56:CA:8F:40,Address_A8:B4:56:CC:DE:00,Address_AC:E2:D3:B6:BA:1E
0,-41,124150763130848,2026,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-49,9138231856000,2026,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-72,702468387488,2026,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-70,70375097565248,2026,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-66,124150763137440,2026,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [0]:
TV_train,TV_test,FV_train,FV_test=train_test_split(TV, FV, test_size=0.1, random_state=1)



---



---


## Actual Accuracy

In [52]:
clf = RandomForestClassifier(max_depth=8,random_state=0)
clf.fit(TV_train,FV_train)
FV_pred = clf.predict(TV_test)
print("actual Accuracy: ", clf.score(TV_test,FV_test)*100)

actual Accuracy:  20.32520325203252


In [0]:
# FV_test.to_numpy()

In [0]:
original = pd.Series([],dtype='float64')
original['x'] = FV_test.to_numpy(copy=True)
original['y'] = FV_test.to_numpy(copy=True)
# original['z'] = FV_test

In [0]:
# original['x'].shape
# type(original)

In [0]:
v = FV_test.to_numpy(copy=True)

In [0]:
# v.size

In [0]:
# original['x']
# FV_test.size

In [0]:
for i in range(0, FV_test.size):
  # original['z'][i] = v[i] %100
  v[i] = v[i] // 100
  original['y'][i] = v[i] %100
  v[i] = v[i] // 100
  original['x'][i] = v[i] %100
  v[i] = v[i] // 100

In [0]:
# original['x'].shape

In [0]:
predicted = pd.Series([],dtype='float64')
predicted['x'] = FV_pred.copy()
predicted['y'] = FV_pred.copy()
# predicted['z'] = FV_pred

In [0]:
for i in range(0,FV_pred.size):
  # predicted['z'][i] = FV_pred[i] %100
  FV_pred[i] = FV_pred[i] // 100
  predicted['y'][i] = FV_pred[i] %100
  FV_pred[i] = FV_pred[i] // 100
  predicted['x'][i] = FV_pred[i] %100
  FV_pred[i] = FV_pred[i] // 100

In [0]:
# predicted['x'].shape

In [0]:
# type(original['x'])

In [0]:
def relaxaCheck(predicted, original):
  relaxed = original.copy()
  for i in range(0, predicted.size):
    if predicted[i] in range(original[i]-2, original[i]+2):
      relaxed[i] = 1
    else:
      relaxed[i] = 0
  
  return relaxed

In [0]:
relaxed_x = relaxaCheck(predicted['x'], original['x'])#.to_numpy())
relaxed_y = relaxaCheck(predicted['y'], original['y'])#.to_numpy())
# relaxed_z = relaxaCheck(predicted['z'], original['z'].to_numpy())

In [0]:
# relaxed_x



---
## Relaxed Accuracy


In [68]:
c=0
for i in range(0, relaxed_x.size):
  if relaxed_x[i] == relaxed_y[i] == 1: #relaxed_z[i] == 1:
    c+=1

print(100*c/relaxed_x.size)

71.54471544715447
